In [ ]:
import numpy as np
import cv2
import imageio
import scipy.ndimage.filters
from skopt import Optimizer
from skopt.space import Integer, Real

# Gaussian filtering array
gaussian = []

for i in range(len(left_images)):
    img_left = left_images[i].astype(float)
    img_right = right_images[i].astype(float)
    I1 = scipy.ndimage.filters.gaussian_filter(img_left, 0.6)
    I2 = scipy.ndimage.filters.gaussian_filter(img_right, 0.6)
    gaussian.append((I1, I2))

# Define the hyperparameter space
space = [Integer(16, 128, name='num_disp_values'),
         Real(0.01, 100, "log-uniform", name='Lambda'),
         Integer(10, 50, name='num_iterations')]

# Objective function to minimize
def objective(num_disp_values, Lambda, num_iterations):

    # Compute the disparity and energy using stereo_bp function
    disparity, energy = stereo_bp(I1, I2, num_disp_values, Lambda, num_iterations=num_iterations)

    # Define the metric you want to minimize, e.g., energy or another error metric
    # Remove single-dimensional entries from the shape of the array
    energy = np.squeeze(energy)

    # If the array has more than one element, return the mean (or another summary statistic)
    if np.size(energy) > 1:
        return np.mean(energy)
    else:
        return np.asscalar(energy)

# Initialize the Bayesian optimizer
opt = Optimizer(space, n_initial_points=10, base_estimator="GP")

# Run the optimization for a certain number of iterations
n_optimization_iterations = 50

for _ in range(n_optimization_iterations):
    # Get the next set of hyperparameters to evaluate
    next_params = opt.ask()

    # Evaluate the objective function using the current hyperparameters
    current_value = objective(*next_params)

    # Update the optimizer with the current objective value and hyperparameters
    opt.tell(next_params, current_value)

# Retrieve the best set of hyperparameters and the corresponding objective value
best_params = opt.Xi[np.argmin(opt.yi)]
best_value = np.min(opt.yi)

print("Best hyperparameters found:")
print("num_disp_values:", best_params[0])
print("Lambda:", best_params[1])
print("num_iterations:", best_params[2])
print("Best value of the objective function:", best_value)
